#### Names of people in the group

Please write the names of the people in your group in the next cell.

Nick Askari

Simen Peder Stang

In [0]:
# Loading modules that we need
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Add your imports below this line

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

# Uncomment if you need
# comments_df = load_df("/user/hive/warehouse/comments")
# badges_df = load_df("/user/hive/warehouse/badges")

#### The Problem: Mining the Interests of Experts

In [0]:
## To-do!
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

variableA = users_df.select('Id', 'Reputation').orderBy(asc('Id'))

variableA.show()


+---+----------+
| Id|Reputation|
+---+----------+
| -1|         1|
|  1|       101|
|  2|       101|
|  3|       101|
|  4|       101|
|  5|       215|
|  6|       101|
|  7|       101|
|  8|       101|
|  9|      1102|
+---+----------+
only showing top 10 rows



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, collect_set, size

# It is 147 rows in posts where PostTypeId = 2 and OwnerUserId = Null

answers = posts_df.filter("PostTypeId = 2")
questions = posts_df.filter("PostTypeId = 1")

# Make aliases to avoid error
answers_alias = answers.alias("ans")
questions_alias = questions.alias("ques")

# Joining answers with questions in order to find the tags
answers_with_tags = answers_alias.join(questions_alias, col("ans.ParentId") == col("ques.Id"), "inner")\
                                 .select(col("ans.OwnerUserId"), col("ques.Tags"))

# Fixing the tag column --> one row for each tag
tags_exploded = answers_with_tags.withColumn("Tags", explode(split(col("tags"), "><")))
tags_cleaned = tags_exploded.withColumn("Tags", regexp_replace(col("Tags"), "^<|>$", ""))

# Count distinct tags
result = tags_cleaned.groupBy("OwnerUserId").agg(countDistinct("Tags").alias("distinct_tags_count"))

# Finally, variableB (removing 'distinct_tags_count' column)
variableB = result.withColumn('Diversity', col('distinct_tags_count') / 638).select('OwnerUserId', 'Diversity').orderBy(desc('Diversity'))

variableB.show()


+-----------+-------------------+
|OwnerUserId|          Diversity|
+-----------+-------------------+
|       1330|0.48746081504702193|
|      64377|0.46551724137931033|
|      45264|0.32445141065830724|
|      14675|0.31347962382445144|
|      80885| 0.3056426332288401|
|      58341|0.30094043887147337|
|      55122|0.28213166144200624|
|      65131| 0.2664576802507837|
|        836| 0.2476489028213166|
|      86339|0.24294670846394983|
|      28175|0.23981191222570533|
|      71442| 0.2335423197492163|
|      38887| 0.2225705329153605|
|      50727| 0.2225705329153605|
|       8878|0.21786833855799373|
|      67328|0.21630094043887146|
|       null|0.21473354231974923|
|        924| 0.2115987460815047|
|      29587| 0.2115987460815047|
|        381|0.21003134796238246|
+-----------+-------------------+
only showing top 20 rows



In [0]:
# We must join variableA and variableB on their respective id's

joined_table = variableA.join(variableB, variableA.Id == variableB.OwnerUserId, how='inner')

correlation = joined_table.stat.corr('Diversity', 'Reputation')

print("The Pearson correlation coefficient between Variable A and Variable B is,\n", correlation)

The Pearson correlation coefficient between Variable A and Variable B is,
 0.721767764862296


### Answers to questions


**Do expert users have specific interests, or do they have general interests?**

The correlation between a users reputation and diversity score is strongly positive. This indicates that expert users (users with high reputation) tend to answer questions related to many topics (many distinct tags), and hence have general interests.

A positive correlation means that the higher the reputation, the higher the number of distinct tags in questions answered. 

